- https://hackernoon.com/learn-blockchains-by-building-one-117428612f46
- http://adilmoujahid.com/posts/2018/03/intro-blockchain-bitcoin-python/
- [Blockchain Demo](https://anders.com/blockchain/blockchain.html)
- [ipynb\_playground/dumbcoin\.ipynb at master · julienr/ipynb\_playground](https://github.com/julienr/ipynb_playground/blob/master/bitcoin/dumbcoin/dumbcoin.ipynb)
- [Explaining blockchain — how proof of work enables trustless consensus](https://keepingstock.net/explaining-blockchain-how-proof-of-work-enables-trustless-consensus-2abed27f0845)
- https://bitcoin.org/bitcoin.pdf

# Walletの実装
- addressで取引先を識別する
- とりあえず今は，Walletは適当な1つの`int`をaddressとして持つことにする

In [43]:
from dataclasses import dataclass
@dataclass
class Wallet:
    address: int

# Transactionの実装
- 取引記録を発行することで，取引を行う

In [44]:
@dataclass
class Transaction:
    sender_address: int
    receiver_address: int
    value: float 

# Ledgerの実装
- transactionを記録しておく台帳を作る
- とりあえずはリストにしておく

In [45]:
Ledger=list

## Demo: シンプルな取引

In [46]:
alice=Wallet(1)
bob=Wallet(2)
ledger=Ledger()

- aliceがtransactionを発行して取引を行う

In [47]:
value=5
transaction = Transaction(1,2,5) # send `value` from Alice to Bob
ledger.append(transaction)

- とりあえずはledgerを第三者に管理してもらうことにする
- 第三者による取引内容の書き換えはどのように防げばよいだろうか

# 公開鍵暗号(電子署名)を用いたtransactionの発行
- senderはtransactionに電子署名を行う
    - 今回はRSAを用いることにする
    - これによって第三者によるtransactionの書き換えを防ぐことができる
    - 電子署名の公開鍵はどのように扱う?
- 公開鍵をWalletのaddressとする!

## transactionの改良
- 電子署名できるようにする
    - 署名field `sign`の追加
    - データを文字列で返す`str_data`の追加

In [50]:
from typing import NamedTuple
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
import json
import dataclasses

In [51]:
@dataclass
class Transaction:
    sender_address: str
    receiver_address: str
    value: float
    sign: str = None
        
    def str_data(self) -> str:
        # return the str of value without `sign`
        d=dataclasses.asdict(self)
        del d["sign"]
        return json.dumps(d)
    
    @property
    def is_signed(self):
        return self.sign is not None

## Walletの改良
- RSAによる公開鍵 - 秘密鍵を実装する
- 簡単のためWalletはaddressを1つのみ持つことにする
    - 例えばBitCoinでは匿名性の向上のため，一つのWalletは複数のキーペアを持つことができる ([参考](https://en.bitcoin.it/wiki/Transaction))

In [52]:
from Crypto.Signature import PKCS1_v1_5
import binascii

In [13]:
def decode_key(key):
    if isinstance(key, str):
        return key
    return binascii.hexlify(key.exportKey(format='DER')).decode('ascii')
def encode_key(key):
    if isinstance(key, RSA._RSAobj):
        return key
    return RSA.importKey(binascii.unhexlify(key))

In [66]:
@dataclass
class Wallet:
    private_key: str = dataclasses.field(init=False)
    address: str = dataclasses.field(init=False)
        
    def __post_init__(self):
        key=RSA.generate(1024)
        self.private_key = decode_key(key)
        self.address = decode_key(key.publickey())
    
    def sign_transaction(self, transaction):
        signer=PKCS1_v1_5.new(encode_key(self.private_key))
        h=SHA.new(transaction.str_data().encode())
        return dataclasses.replace(transaction, sign=signer.sign(h).hex())

In [67]:
def verify_transaction(transaction):
    if transaction.sign is None:
        return False
    h=SHA.new(transaction.str_data().encode())
    verifier=PKCS1_v1_5.new(encode_key(transaction.sender_address))
    return verifier.verify(h, binascii.unhexlify(transaction.sign))

## Demo: RSAを用いたtransactionの発行とチェック

In [68]:
alice=Wallet() # now the address is automatically generated
bob=Wallet()

In [69]:
value=5
transaction=alice.sign_transaction(Transaction(alice.address, bob.address, value))

- check

In [70]:
verify_transaction(transaction)

True

- 第三者によるデータの改竄リスクを低減することができた 
- しかし依然として，double-spending problemを防ぐには信頼のある第三者が必要となる
- すべてのtransactionが公開されていれば，double-spendingは防げるのでは？

# Block ChainによるTransactionの公開
- いくつかのtransactionをまとめて，timestamp付き`block`で公開することを考える
    - ある特定の時間にそのtransactionが存在したことが保証される
- さらに，blockには直前のtimestampのhashを含めることにする
    - 直前の`block`が強化されることになる
- これでtransactionをチェックしてdouble-spendingを防ぐ第三者は必要なくなった
    - 今度は信頼のあるtimestampサーバが必要になったが，これは後ほど解決する

## Blockの実装
- ついでにhash関数も実装する

In [71]:
from typing import Tuple
import hashlib

In [73]:
@dataclass
class Block:
    time: float
    transactions: Tuple[Transaction]
    previous_hash: str
        
def hash_block(block: Block): 
    block_bytes=json.dumps(block).encode()
    return hashlib.sha256(block_bytes).hexdigest()

## BlockChainの実装
- 公開されたBlockのhashをチェックし，chain状にBlockを保持する
- 最初のblockはprevious blockがないため，特別なブロック(genesis)を用意する
- Genesis Block
    - https://en.bitcoin.it/wiki/Genesis_block
    - BitCoinでは，ハードコードされている
    - addressも書いてあるが，そのprivate keyをSatoshi Nakamotoが持っているかは不明らしい
- 今回はただのリストにする

In [74]:
from time import time

In [75]:
def verify_block(previous_block, block) -> bool:
    is_correct_hash = hash_block(previous_block) ==  block.previous_hash
    is_correct_transactions = all(map(verify_transaction, block.transactions))
    return is_correct_hash and is_correct_transactions

## Demo: Blockの公開とBlockChainへの登録

In [76]:
genesis=Block(time(), (), "0")
block_chain=[genesis]

- 取引

In [78]:
transactions=[]
transactions.append(alice.sign_transaction(Transaction(alice.address, bob.address, 5)))
transactions.append(bob.sign_transaction(Transaction(bob.address, alice.address, 10)))

- Block生成

In [80]:
json.dumps(transactions[0])

TypeError: Object of type Transaction is not JSON serializable

In [79]:
previous_hash=hash_block(block_chain[-1])
block=Block(time(), tuple(transactions), previous_hash)

TypeError: Object of type Block is not JSON serializable

- block chainへの登録

In [25]:
verify_block(block_chain[-1], block)

True

In [26]:
block_chain.append(block)

- 2回目は登録に失敗する

In [27]:
verify_block(block_chain[-1], block)

False

- timestampサーバを第三者に任せない方法がほしい
- そもそもBlockの生成はだれがやるのか?

# Proof-of-Workの実装
- これがblockchainの革新的技術
- block chainの正当性をtrust依存にするのではなく，cryptograph proofに基づくものにする
    - chainの暗号学的強度を高める
    - 同時にどのchainが"正しい"のか，決定するアルゴリズムも与える
- blockの生成を"コストの高い作業"することによって強度を高める
    - この作業によって生成された値`nonce`はblockの正当性証明となる
    - blockは1つ前のhashも持っているから，chainの途中のblockを改ざんするには，そこから先すべてのblockを改ざんする必要がある
    - blockの生成は計算コストが非常に高いから，chainの改ざんは莫大なコストを要することになる
- 最も長いchainを"正しい"chainとする
    - 最も長いchainは，もっとも多くの計算資源を投入していることになる
    - これはone-CPU-one-voteとみなせる

## blockにnonceを加える
- blockのハッシュ値が適当な条件を満たすように`nonce`を追加する
    - 例えば上4桁が0
    - この制約を"difficulty"と呼ぶ
    - difficultyは適当に調整される. 例えばbitcoinの場合は，blockの追加のintervalが10分程度になるように調整されるようだ
        - [Difficulty \- Bitcoin Wiki](https://en.bitcoin.it/wiki/Difficulty)
    - 今回はdifficultyは固定にする
- nonceを見つける作業が"mine"
    - nonceが正しいことを確認するのは簡単
    - nonceを見つけるのは難しい(総当たりで試して，あたりを引くしかない)
    - これによって"block"の生成を非常に高コストにすることができる
    - と同時に，正当性の確認にもなる
- "mine"は第三者が行う

In [28]:
class Block(NamedTuple):
    time: float
    transactions: Tuple[Transaction]
    previous_hash: str
    nonce: int = None

## mineの実装

In [29]:
difficulty=4
def valid_proof(block):
    return hash_block(block)[:difficulty] == "0" * difficulty

def mine(block):
    nonce=0
    block=block._replace(nonce=nonce)
    while not valid_proof(block):
        nonce += 1
        block=block._replace(nonce=nonce)
    return block

- ついでにverify_blockも改良

In [30]:
def verify_block(previous_block, block) -> bool:
    is_correct_hash = hash_block(previous_block) ==  block.previous_hash
    is_correct_transactions = all(map(verify_transaction, block.transactions))
    is_correct_proof = valid_proof(block)
    return is_correct_hash and is_correct_transactions and is_correct_proof

## Demo4: tranasctionの発行からminingまで

In [31]:
genesis=Block(time(), (), "0")
block_chain=BlockChain(genesis)

# 取引
transactions=[]
transactions.append(alice.sign_transaction(Transaction(alice.address, bob.address, 5)))
transactions.append(bob.sign_transaction(Transaction(bob.address, alice.address, 10)))

- mine

In [32]:
previous_hash=hash_block(block_chain[-1])
block=mine(Block(time(), tuple(transactions), previous_hash))

In [33]:
block.nonce

99021

In [34]:
verify_block(block_chain[-1], block)

True

# NodeとNetwork
- 前回までで分散台帳の準備は整った
- 具体的にblockchainを保持する`node`を実装する
- 具体的なnetworkの動きは以下のようになる ([@Nakamoto2018](www.bitcoin.org))

1. transaction生成．なるべく多くのnodeにbroadcastする
2. nodeはtransactionを集めてblockにまとめる
3. mine
4. mineが終わったら，他のnodeにblockをbroadcastする
5. blockを受け取ったnodeは，blockの正当性を確認し，double-spendingのチェックをする
6. チェックが通れば，次のblockの作成を始める

- ついでにいままでのいろいろなmethodをまとめる
- 今回はbroadcastはmultiprocessのサーバープロセスを用いて模擬する
- データのやり取りはすべてjson

In [35]:
from uuid import uuid4
from copy import deepcopy
import multiprocessing as mp

In [36]:
class Network:
    def __init__(self):
        self.manager=mp.Manager()
        self.chains=self.manager.dict()
        self.blocks=self.manager.dict()
        self.transactions=self.manager.dict()
        
    def add_node(self, uuid, chain=()):
        self.chains[uuid]=chain
        self.blocks[uuid]=self.manager.list()
        self.transactions[uuid]=self.manager.list()
    
    def post_chain(self, uuid, chain):
        self.chains[uuid]=tuple(chain)
    
    def get_chain(sef, uuid):
        return self.chains[uuid]
    
    def post_block(self, block):
        for k, v in self.blocks.items():
            v.append(block)
            
    def get_blocks(self, uuid):
        l=list(self.blocks[uuid])
        self.blocks[uuid][:]=[]
        return l
    
    def post_transaction(self, transaction):
        for k, v in self.transactions.items():
            v.append(transaction)
            
    def get_blocks(self, uuid):
        l=list(self.transactions[uuid])
        self.transactions[uuid][:]=[]
        return l
    
    def shutdown(self):
        self.manager.shutdown()

In [37]:
def encode_transction(transaction):
    return json.dumps(transction)
def decode_transaction(transaction_json):
    return Transaction(*json.loads(transaction_json))

def encode_block(block):
    return json.dumps(block)
def decode_block(block_json):
    block=Block(*json.loads(block_json))
    return block._replace(transactions=tuple(map(lambda x: Transaction(*x), block.transactions)))

def encode_chain(chain):
    return json.dumps(chain)
def decode_chain(chain_json):
    return [decode_block(json.dumps(Block(*l))) for l in json.loads(chain_json)]

In [38]:
class Node:
    def __init__(self, network):
        self.chain=[]
        self.uuid=str(uuid4())
        self.network=network
        network.add_node(self.uuid)
        
    @staticmethod
    def hash_block(block): 
        block_bytes=json.dumps(block).encode()
        return hashlib.sha256(block_bytes).hexdigest()
        
    @staticmethod
    def valid_proof(block):
        return self.hash_block(block)[:difficulty] == "0" * difficulty

    def mine(self):
        transactions=self.network.transactions[self.uuid]
        if len(transactions)==0:
            return False # cannot mine due to no transactions
        previous_hash=self.hash_block(self.chain[-1])
        block=Block(time(), tuple(broadcasted_transactions), previous_hash)
        
        nonce=0
        block=block._replace(nonce=nonce)
        while not valid_proof(block):
            nonce += 1
            block=block._replace(nonce=nonce)
        self.broadcast_block(block)
        return True # successfully mined
        
    def add_block(self):
        for block in broadcasted_blocks:
            if verify_block(block):
                self.chain.append(block)
                return True 
        return False # no block is added
    
    def broad_cast(self):
        broadcasted_blocks.append(self.chain[-1])
    
    def publish_chain(self):
        chains[self.uuid]=self.chain
        
        
    @staticmethod
    def verify_transaction(transaction):
        if transaction.sign is None:
            return False
        h=SHA.new(transaction.str_data().encode())
        verifier=PKCS1_v1_5.new(encode_key(transaction.sender_address))
        return verifier.verify(h, binascii.unhexlify(transaction.sign))
    
    @staticmethod
    def verify_block(block) -> bool:
        previous_block=self.chain[-1]
        is_correct_hash = self.hash_block(previous_block) ==  block.previous_hash
        is_correct_transactions = all(map(self.verify_transaction, block.transactions))
        is_correct_proof = self.valid_proof(block)
        return is_correct_hash and is_correct_transactions and is_correct_proof
    
    @staticmethod
    def valid_chain(chain):
        for i in range(len(chain)-1, -1, -1):
            if not chain.valid_block(chain[i]):
                return False
            if i > 0 and chain[i-1].hash != chain[i].previous_hash:
                return False
        return True
    
    def resolve_conflicts(self):
        """Longest valid chain is authoritative"""
        authoritative_chain=self.chain
        for node_id in self.neighbours:
            node=nodes[node_id]
            if not self.valid_chain(node.chain):
                # node is incorrect
                continue
            if len(node.chain)>len(authoritative_chain):
                # Longest valid chain is authoritative
                authoritative_chain=deepcopy(node.chain)
        self.chain=authoritative_chain